**Initialization**

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, col


**Read Bronze Table**

In [0]:
df = spark.read.table("workspace.bronze.erp_px_cat_g1v2")


**Silver Transformation**

**Trim all String values**

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))
     


**Normalization Maintenance Flag to Boolean**

In [0]:
df = df.withColumn(
    "maintenance",
    F.when(F.upper(col("maintenance")) == "YES", F.lit(True))
     .when(F.upper(col("maintenance")) == "NO", F.lit(False))
    .otherwise(None)
)

**Renaming Columns**

In [0]:
df = (
    df
    .withColumnRenamed("id", "category_id")
    .withColumnRenamed("cat", "category")
    .withColumnRenamed("subcat", "subcategory")
    .withColumnRenamed("maintenance", "maintenance_flag")
   
)

**Sanity checks of dataframe**

In [0]:
df.limit(10).display()

**Writing Silver Table**

In [0]:
df.write \
  .mode("overwrite") \
  .format("delta") \
  .saveAsTable("workspace.silver.erp_product_category")

**Sanity checks of silver table**

In [0]:
%sql
SELECT * FROM WORKSPACE.silver.erp_product_category LIMIT 10;